In [45]:
import random
import numpy as np
import matplotlib.pyplot as plt
import copy

In [51]:
class Event:
    def __init__(self):
        self.label = 2
        self.start = -1
        self.end = -1

class SegmentationClass:
    def __init__(self):
        self.labels = None

    def event_detection(self, labels):
        print(labels)
        self.labels = [2]
        self.labels += labels.tolist() # first frame becomes a dummy 
        print(self.labels)
        
        
        prev_event = self.labels[0]
        events = []
        event = Event()
        event_in_progress = False
        self.dilation()
        print(self.labels)
        
        i = 1
        while i < len(self.labels):
            event.label = prev_event
            event.start = i
            while self.labels[i] == prev_event:
                i+=1
                if i >= len(self.labels):
                    break
            event.end = i-1
            events.append(copy.deepcopy(event))
            if i >= len(self.labels):
                break
            prev_event = self.labels[i]

        final_events = self.prune_events(events)
        return final_events
    
    def prune_events(self, events, thresh=3):
        final_events = []
        for e in events:
#             print("pruning events")
            if e.end - e.start + 1 >= thresh and e.label != 2:
#                 print(f'e.end = {e.end} and e.start = {e.start}')
                final_events.append(e)
        return final_events
    
    def dilation(self, window=5):
        offset = int(window/2)
        L = len(self.labels)
        i = 1 + offset
        new_labels = copy.deepcopy(self.labels)
        
        while i < L - offset -1:
            count = [0,0,0]
            j = i - offset
            while j <= i + offset:
                count[self.labels[j]] +=1 
                j+=1
            new_labels[i] = count.index(max(count))
            i+=1
        self.labels = new_labels

In [47]:
predicted = np.random.randint(low=0, high=3, size=20)

In [48]:
print(predicted)

[1 2 1 1 2 2 0 0 2 2 1 1 2 2 0 0 0 0 0 1]


In [52]:
segmentationClass = SegmentationClass()

In [53]:
events = segmentationClass.event_detection(predicted)

[1 2 1 1 2 2 0 0 2 2 1 1 2 2 0 0 0 0 0 1]
[2, 1, 2, 1, 1, 2, 2, 0, 0, 2, 2, 1, 1, 2, 2, 0, 0, 0, 0, 0, 1]
[2, 1, 2, 1, 2, 1, 0, 2, 2, 0, 1, 2, 2, 1, 0, 0, 0, 0, 0, 0, 1]


In [54]:
for e in events:
    print(f'{e.label} : {e.start} - {e.end}')

0 : 14 - 19


In [125]:
final_events = segmentationClass.prune_events(events, 3)

e.end = 10 and e.start = 8
e.end = 15 and e.start = 12


In [126]:
for e in final_events:
    print(f'{e.label} : {e.start} - {e.end}')

0 : 8 - 10
0 : 12 - 15
